<a href="https://colab.research.google.com/github/rajeevfromkrec/bert/blob/master/Bert_classification_in_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers

     |████████████████████████████████| 890kB 4.2MB/s 
     |████████████████████████████████| 890kB 14.0MB/s 
     |████████████████████████████████| 1.1MB 22.3MB/s 
     |████████████████████████████████| 3.0MB 32.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=91497621b36739df8bf90855f5b62787b52810527c0980086153cd1b84201d41
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [6]:
import numpy as np
import pandas as pd
import torch
import transformers as pbb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression


In [8]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

In [10]:
df.head()

,0,1
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1


In [11]:
batch_1=df[:2500]

In [13]:
batch_1[1].value_counts()

1    1293
0    1207
Name: 1, dtype: int64

In [14]:
model=pbb.DistilBertModel
tokenizer=pbb.DistilBertTokenizer

In [16]:
tokenizer=tokenizer.from_pretrained('distilbert-base-uncased')

In [17]:
model=model.from_pretrained('distilbert-base-uncased')

In [18]:
tokenized=batch_1[0].apply(lambda x: tokenizer.encode(x,add_special_tokens=True))

In [20]:
type(tokenized)

pandas.core.series.Series

In [22]:
print(tokenized[0])

[101, 1037, 18385, 1010, 6057, 1998, 2633, 18276, 2128, 16603, 1997, 5053, 1998, 1996, 6841, 1998, 5687, 5469, 3152, 102]


In [24]:
print(batch_1[0][0])

a stirring , funny and finally transporting re imagining of beauty and the beast and 1930s horror films


In [25]:
max_len=0

for i in tokenized.values:
  if len(i)>max_len:
    max_len=len(i)

In [26]:
max_len

65

In [28]:
padded= np.array([i+[0]*(max_len-len(i)) for i in tokenized.values])

In [29]:
padded[0]

array([  101,  1037, 18385,  1010,  6057,  1998,  2633, 18276,  2128,
       16603,  1997,  5053,  1998,  1996,  6841,  1998,  5687,  5469,
        3152,   102,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0])

In [30]:
padded.shape

(2500, 65)

In [31]:
attention_mask=np.where(padded !=0,1,0)

In [32]:
attention_mask.shape

(2500, 65)

In [33]:
input_ids=torch.tensor(padded)
attention_mask=torch.tensor(attention_mask)

In [34]:
with torch.no_grad():
  last_hidden_state=model(input_ids,attention_mask=attention_mask)

In [38]:
last_hidden_state[0].shape

torch.Size([2500, 65, 768])

In [40]:
features=last_hidden_state[0][:,0,:].numpy()

In [42]:
labels=batch_1[1]

In [43]:
train_x,test_x,train_y,test_y= train_test_split(features,labels  )

In [44]:
lr_clf=LogisticRegression()
lr_clf.fit(train_x,train_y)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [45]:
lr_clf.score(test_x,test_y)

0.824